In [ ]:
# Before beginning this notebook you will need to add the following code to the query tool in pgAdmin: 
# ALTER TABLE counties ADD PRIMARY KEY (fips_date); ALTER TABLE hospitals ADD PRIMARY KEY (fips_date); ALTER TABLE vaccinations ADD PRIMARY KEY (fips_date);
# Additionally, you will need to enter your password into the connection_string variable in the config.py file. 
# The place holder text is YOURPASSWORDHERE

In [6]:
# Import dependencies
import sqlalchemy
from sqlalchemy import create_engine
import config
import pandas as pd
import psycopg2 as pg
import numpy as np

In [7]:
 # Create engine using the database file
engine = sqlalchemy.create_engine(config.connection_string)

In [8]:
# Create the connection
dbConnection = engine.connect();

In [9]:
# Join counties and hospitals tables into dataframe
merged_df = pd.read_sql("SELECT * FROM counties JOIN hospitals ON counties.fips_date = hospitals.fips_date", dbConnection);

In [10]:
# Create dataframe of vaccinations table
vaccinations_df = pd.read_sql("SELECT * FROM vaccinations", dbConnection);

ProgrammingError: (psycopg2.errors.UndefinedTable) 오류:  "vaccinations" 이름의 릴레이션(relation)이 없습니다
LINE 1: SELECT * FROM vaccinations
                      ^

[SQL: SELECT * FROM vaccinations]
(Background on this error at: http://sqlalche.me/e/14/f405)

In [ ]:
# Create a list of the dataframes
dfs = [merged_df, vaccinations_df]

In [ ]:
merged_df

In [ ]:
for col in merged_df.columns:
    print(col)

In [ ]:
column_numbers = [x for x in range(merged_df.shape[1])]

In [ ]:
column_numbers .remove(0) #removing column integer index 0
merged_df = merged_df.iloc[:, column_numbers] #return all columns except the 0th column
merged_df

In [ ]:
column_numbers = [x for x in range(merged_df.shape[1])]
column_numbers .remove(2) #removing column integer index 0
merged_df = merged_df.iloc[:, column_numbers] #return all columns except the 0th column
merged_df

In [ ]:
column_numbers = [x for x in range(merged_df.shape[1])]
column_numbers .remove(2) #removing column integer index 0
merged_df = merged_df.iloc[:, column_numbers] #return all columns except the 0th column
merged_df

In [ ]:
for col in merged_df.columns:
    print(col)

In [ ]:
# Create ratio columns for bed usage (ICU and inpatient)

In [ ]:
for col in vaccinations_df.columns:
    print(col)

In [ ]:
ml_ready_df = merged_df.merge(vaccinations_df, how='inner', on='fips_date')
ml_ready_df

In [ ]:
for col in ml_ready_df.columns:
    print(col)

In [ ]:
ml_ready_df = ml_ready_df.drop('index', 1)
ml_ready_df

In [ ]:
ml_ready_df["Percentage_inpatient_beds_used"] = ml_ready_df["inpatient_beds_used_7_day_sum"]/ml_ready_df["inpatient_beds_7_day_sum"]
ml_ready_df

In [ ]:
ml_ready_df["Percentage_ICU_beds_used"] = ml_ready_df["icu_beds_used_7_day_sum"]/ml_ready_df["total_icu_beds_7_day_sum"]
ml_ready_df

In [ ]:
ml_ready_df["total_patients_hospitalized_con_and_sus_covid_7_day_sum"] = ml_ready_df["total_adult_patients_hospitalized_con_and_sus_covid_7_day_sum"]+ml_ready_df["total_ped_patients_hospitalized_con_and_sus_covid_7_day_sum"]
ml_ready_df

In [ ]:
ml_ready_df["total_patients_hospitalized_con_covid_7_day_sum"] = ml_ready_df["total_adult_patients_hospitalized_con_covid_7_day_sum"]+ml_ready_df["total_ped_patients_hospitalized_con_covid_7_day_sum"]
ml_ready_df

In [ ]:
ml_ready_df.replace([np.inf, -np.inf], np.nan)

In [ ]:
ml_ready_df.dropna()

In [ ]:
ml_ready_df.dtypes

In [ ]:
ml_ready_df = ml_ready_df.astype({'fips_code': 'int64'})

In [ ]:
ml_ready_df.dtypes

In [ ]:
ml_ready_df.sort_values(by=['fips_date'], inplace=True)

Data Exploration Charts

In [ ]:
multnomah_df = ml_ready_df[ml_ready_df["fips_code"] == 41051]
multnomah_df

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
multnomah_df.plot.bar(x="collection_week", y="Percentage_inpatient_beds_used", figsize=(20, 20))
plt.ylim(top=1.0)
plt.ylim(bottom=0.4)
plt.savefig('Percentage of Inpatient Beds Used Multnomah County')
plt.show()

In [ ]:
multnomah_df.plot.bar(x="collection_week", y="cases_to_date", figsize=(20, 20))
plt.savefig('Cases to Date Multnomah County')
plt.show()

In [ ]:
multnomah_df.plot.bar(x="collection_week", y="deaths_to_date", figsize=(20, 20))
plt.savefig('Deaths to Date Multnomah County')
plt.show()

In [ ]:
multnomah_df.plot.bar(x="collection_week", y="Series_Complete_Pop_Pct", figsize=(20, 20))
plt.savefig('Percent of Population Fully Vaccinated Multnomah County')
plt.show()

# ML_model

In [11]:
at_capacity = []
for value in ml_ready_df['Percentage_ICU_beds_used']:
    if value >0.85:
        at_capacity.append("1")
    else :
        at_capacity.append("0")
ml_ready_df["at_capacity"] = at_capacity
ml_ready_df["at_capacity"].astype(str).astype(float)
ml_ready_df

NameError: name 'ml_ready_df' is not defined

In [ ]:
import numpy as np
ml_ready_df = ml_ready_df.replace([np.inf, -np.inf], np.nan)
ml_ready_df.dropna(inplace=True)

In [ ]:
ml_ready_df.drop(['fips_date', 'fips_code', 'collection_week'], axis=1, inplace=True)
ml_ready_df

In [ ]:
for column in ml_ready_df.columns:
    print(f"column {column} has {ml_ready_df[column].isnull().sum()} null values")

In [ ]:
ml_ready_df = ml_ready_df.dropna()

In [ ]:
for column in ml_ready_df.columns:
    print(f"column {column} has {ml_ready_df[column].isnull().sum()} null values")

In [ ]:
ml_ready_df.duplicated().sum()

In [ ]:
ml_ready_df.dtypes

In [ ]:
np.round(ml_ready_df['Percentage_ICU_beds_used'],2)

In [ ]:
s_df = ml_ready_df.sample(3500)

In [ ]:
# Create the model variables
X = s_df.drop(['at_capacity'], axis=1)
y = s_df['at_capacity']

In [ ]:
# Split, Test, and Train the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=42,stratify=y)

In [ ]:
# Create the logistic regression model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

In [ ]:
# use fit() to train the logistic regression model
classifier.fit(X_train, y_train)

In [ ]:
# Make predictions of the data
predictions = classifier.predict(X_test)

In [ ]:
# Get the accuracy score of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

In [ ]:
# Print the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, predictions)
print(matrix)

In [ ]:
# Get the classification report
report = classification_report(y_test, predictions)
print(report)

# Random OverSample

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
from collections import Counter
Counter(y_resampled)

In [ ]:
# Rerun the model
model = LogisticRegression(solver='lbfgs', random_state=42)

model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))

from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(y_test, y_pred))

In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

# SMOTE Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)
Counter(y_resampled)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
bed_capacity = balanced_accuracy_score(y_test, y_pred)
bed_capacity

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Create df for confusion matrix
cm_df = pd.DataFrame(cm, index=["Actual full", "Actual spacious"], columns=["Predicted full", "Predicted spacious"])

cm_df

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {bed_capacity}")
print("Classification Report")
print(classification_report(y_test, y_pred))

In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

# K-Means 

In [ ]:
# Initial imports
import pandas as pd
from sklearn.cluster import KMeans
import plotly.express as px
import hvplot.pandas

In [ ]:
s_df = ml_ready_df.sample(7000)

In [ ]:
s_df.hvplot.scatter(x="Completeness_pct", y="Percentage_ICU_beds_used")

In [ ]:
# Function to cluster and plot dataset
def test_cluster_amount(df, clusters):
    model = KMeans(n_clusters=clusters, random_state=5)
    model

    # Fitting model
    model.fit(df)

    # Add a new class column to ml_ready_df
    df["class"] = model.labels_

In [ ]:
test_cluster_amount(s_df, 4)
s_df.hvplot.scatter(x="Completeness_pct", y="Percentage_ICU_beds_used", by="class")

In [ ]:
fig = px.scatter_3d(
    s_df,
    x="Completeness_pct",
    y="Percentage_ICU_beds_used",
    z="cases_to_date",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()